In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import dummy
import baseline
import fixed
import correlation
import bayesian

import utils as u

def clip(x):
    return max(1e-5, min(1-1e-5, x))

print("Reading file . . .")
DATA = pd.read_csv("./data/induce-data-2019-08-08.csv",index_col=0)
print("I'm so tired . . .")

# DATA = DATA[DATA.topic == "animals"]

USERS = DATA.user.unique() 
FOLDS = [np.random.randint(10) for _ in USERS]

for fold in range(1):
    print("&&&" * 20)
    print()
    print("FOLD", fold)
    print()

    TRAIN_USER = [u for i,u in enumerate(USERS) if FOLDS[i] % 3 == 0]
    TEST__USER = [u for i,u in enumerate(USERS) if FOLDS[i] == fold]
    
    TRAIN_DATA = DATA[DATA.user.isin(TRAIN_USER)]
    TEST__DATA = DATA[DATA.user.isin(TEST__USER)]
    
    models = [bayesian.Model(12)]
    for model in models:
        
        print("--" * 20)
        print()
        print("Training Model   :", model.name)
        model.train(TRAIN_DATA)
        
        print("***" * 20)

        print("Evaluating Model :", model.name)
        LOG_ERR = 0
        MSE_ERR = 0
        ACC_ERR = 0

        for user in TEST__USER:
            userSequence = [x[1] for x in TEST__DATA[TEST__DATA.user == user].iterrows()]
            for idx in range(len(userSequence)):

                [proba, prediction] = model.predict(userSequence[:idx], userSequence[idx])

                if prediction not in u.accepted_answers[userSequence[idx].question]:
                    raise Exception("Predictor incorrectly assessed possible answers")

                p = clip(proba)
                
                LOG_ERR -= np.log(p) if prediction == userSequence[idx].result else np.log(1-p)
                MSE_ERR += (1-p) ** 2 if prediction == userSequence[idx].result else p ** 2
                ACC_ERR += 1 if (prediction == userSequence[idx].result) == (p >= 0.5) else 0

        LOG_ERR /= len(TEST__DATA)
        MSE_ERR /= len(TEST__DATA)
        ACC_ERR /= len(TEST__DATA)
        
        print()
        print("LOG_ERR = %.3f" % LOG_ERR)
        print("MSE_ERR = %.3f" % MSE_ERR)
        print("ACC_ERR = %.3f" % ACC_ERR)
        print()

SyntaxError: invalid syntax (<ipython-input-1-a48d0d53df76>, line 31)